# Import

In [5]:
# import all packages
import pandas as pd
import numpy as np
import psycopg2 as psycopg2
import sql_functions as sqlf
import string as string

In [2]:
# pd.set_option("display.max.rows", 150)

In [3]:
schema = "capstone_24_4_group1"

In [4]:
bx_query = f'''   SELECT *
                    FROM {schema}."raw_scraped_box_office"
                    '''

In [5]:
box_office_df = sqlf.get_dataframe(bx_query)

In [ ]:
display(box_office_df.head(10))

In [7]:
# Transformation of colum [value]
box_office_df.loc[ : , "value"] = box_office_df["value"].str.replace("$", "")
box_office_df.loc[ : , "value"] = box_office_df["value"].str.replace(",", "")
box_office_df["value"] = box_office_df["value"].astype("Int64")

# Transformation of colum [release_group]
box_office_df["release_group"] = box_office_df["release_group"].astype("Int64")

In [8]:
box_office_df.dropna(inplace= True)

### Checked: Filme mit Film- und re-Release (bezogen auf Domestic-Daten) Lifetime domestic Daten haben

In [9]:
box_office_df.drop(box_office_df[((box_office_df["release_group"] == 0) | (box_office_df["release_group"] == 1)) & (box_office_df["region"] == "Domestic")].index, inplace = True)

In [ ]:
box_office_df

In [11]:
bx_df_new = box_office_df[(box_office_df["release_group"] == 0) | (box_office_df["release_group"] == 1)].groupby(["tconst", "region"]).sum()
bx_df_new["release_group"] = 2
bx_df_new.reset_index(inplace=True)

In [12]:
box_office_df = pd.concat([box_office_df,bx_df_new])
box_office_df.drop(box_office_df[(box_office_df["release_group"] == 0) | (box_office_df["release_group"] == 1)].index, inplace = True)
box_office_df.drop(columns="release_group", inplace = True)

## Inflation correction

In [13]:
# Defining SQL queries for download
imdb_tickets_data = f'''   SELECT *
                    FROM {schema}."IMDB_tickets_data"
                    '''

In [14]:
imdb_df = sqlf.get_dataframe(imdb_tickets_data)

In [ ]:
display(imdb_df.head(10))

In [16]:
# Merge sales AND year data to Boxoffice-table
box_office_df = box_office_df.merge(imdb_df[["tconst", "year", "NA_gross_sales"]], how = "inner", left_on = "tconst", right_on = "tconst")

# Resort columns
box_office_df = box_office_df.reindex(["tconst","region", "year", "value", "NA_gross_sales"], axis=1)

In [ ]:
# Import inflation correction function
import cpi
# cpi.update()

In [19]:
box_office_df.drop(box_office_df[box_office_df["year"] == 1997].index, inplace = True)

In [ ]:
unique_years = box_office_df['year'].unique()
inflation_factors = {year: cpi.inflate(1, year, to=2023, items="Admission to movies, theaters, and concerts") for year in unique_years}

box_office_df['inflation_factor'] = box_office_df['year'].map(inflation_factors)

box_office_df['inflated_values'] = box_office_df['value'] * box_office_df['inflation_factor']

In [ ]:
display(box_office_df)

In [22]:
unique_years = box_office_df['year'].unique()
inflation_factors = {year: cpi.inflate(1, year, to=2023, items="Admission to movies, theaters, and concerts") for year in unique_years}

box_office_df['NA_inflation_factor'] = box_office_df['year'].map(inflation_factors)

box_office_df['NA_inflated_values'] = box_office_df['NA_gross_sales'] * box_office_df['NA_inflation_factor']

In [ ]:
display(box_office_df)

In [24]:
box_office_df.drop(columns=['inflation_factor', 'NA_inflation_factor'], inplace=True)

In [ ]:
box_office_df.shape

BOXOFFICE-TABELLE AN DIESER STELLE BEREIT FÜR UPLOAD (so wie damals ...)

In [27]:
PIVOT_box_df = box_office_df.pivot(index='tconst', columns='region', values=['inflated_values', 'value']).reset_index()

In [ ]:
PIVOT_box_df

## UPLOAD

In [31]:
dataframes = {
    "box_office_TEMP_PIVOT":PIVOT_box_df
}

In [ ]:
# table_name = 'eu_merged'
engine = sqlf.get_engine()
engine
# Write records stored in a dataframe to SQL database
if engine != None:
    try:
        for table_name, df in dataframes.items():
            df.to_sql(table_name, # Name of SQL table
                            con=engine, # Engine or connection
                            if_exists='replace', # Drop the table before inserting new values 
                            schema=schema, # your class schema
                            index=False, # Write DataFrame index as a column
                            chunksize=5000, # Specify the number of rows in each batch to be written at a time
                            method='multi') # Pass multiple values in a single INSERT clause
            print(f"The {table_name} table was uploaded successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
else:
    print("nope") 

### Zwischenschritt: In Jonas' branch. Jupyter Notebook "combine_box_office_worldwide"

# MELTING & COUNTRY CORRECTION

## DOWNLOAD

In [2]:
schema = "capstone_24_4_group1"

In [3]:
bx_inf_query = f'''   SELECT *
                    FROM {schema}."box_office_data_inflated"
                    '''

In [6]:
box_inflated = sqlf.get_dataframe(bx_inf_query)

In [7]:
box_inflated.head(5)

,tconst,country,values,inflated_values
0,tt7399138,Albania,483.0,566.208970
1,tt10223460,Albania,30528.0,32280.176916
2,tt1051906,Albania,3359.0,3874.880639
3,tt10665342,Albania,9906.0,10474.562124
4,tt10954984,Albania,4186.0,4426.258535


In [8]:
box_inflated["country"].unique()

array(['Albania', 'Argentina', 'Aruba', 'Australia', 'Austria', 'Bahrain',
       'Baltic states', 'Bangladesh', 'Belgium', 'Bolivia', 'Bosnia',
       'Bosnia and herzegovina', 'Brazil', 'Bulgaria', 'Cambodia',
       'Canada', 'Central america', 'Chile', 'Central america+', 'China',
       'Colombia', 'Costa rica', 'Croatia', 'Curaçao', 'Cyprus',
       'Czech republic', 'Czech republic/slovakia', 'Denmark', 'Djibouti',
       'Domestic', 'Dominican republic', 'E/w africa', 'East africa',
       'Ecuador', 'Egypt', 'El salvador', 'Estonia', 'Ethiopia',
       'Finland', 'France', 'Germany', 'Ghana', 'Greece',
       'Greece and cyprus', 'Guatemala', 'Honduras', 'Hong kong',
       'Hungary', 'Iceland', 'India', 'Indonesia', 'Iraq', 'Ireland',
       'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan', 'Kenya', 'Kuwait',
       'Laos', 'Latvia', 'Lebanon', 'Lebanon/united arab emirates',
       'Lesser antilles', 'Lithuania', 'Malaysia', 'Malta', 'Mauritius',
       'Mexico', 'Middle east

In [9]:
bx_csv = pd.read_csv("Data/bx_countries.csv", sep=";")

In [10]:
display(bx_csv.head(3))

,bx_country,long_name
0,Albania,Albania
1,Argentina,Argentina
2,Aruba,Aruba


In [11]:
bx_infl_new = box_inflated.merge(bx_csv, how= "left", left_on="country", right_on="bx_country")

In [12]:
bx_infl_new["long_name"].unique()

array(['Albania', 'Argentina', 'Aruba', 'Australia', 'Austria', 'Bahrain',
       'Baltic states', 'Bangladesh', 'Belgium',
       'Plurinational State of Bolivia', 'Bosnia and Herzegovina',
       'Brazil', 'Bulgaria', 'Cambodia', 'Canada', 'Central America',
       'Chile', 'China', 'Colombia', 'Costa Rica', 'Croatia', 'Curaçao',
       'Cyprus', 'Czechia', 'Czech Republic/Slovakia', 'Denmark',
       'Djibouti', 'United States of America', 'Dominican Republic',
       'E/W Africa', 'East Africa', 'Ecuador', 'Egypt', 'El Salvador',
       'Estonia', 'Ethiopia', 'Finland', 'France', 'Germany', 'Ghana',
       'Greece', 'Greece and Cyprus', 'Guatemala', 'Honduras',
       'Hong Kong', 'Hungary', 'Iceland', 'India', 'Indonesia', 'Iraq',
       'Ireland', 'Israel', 'Italy', 'Jamaica', 'Japan', 'Jordan',
       'Kenya', 'Kuwait', 'Laos', 'Latvia', 'Lebanon',
       'Lebanon/United Arab Emirates', 'Netherlands Antilles',
       'Lithuania', 'Malaysia', 'Malta', 'Mauritius', 'Mexico',
     

In [14]:
bx_infl_new

,tconst,country,values,inflated_values,bx_country,long_name
0,tt7399138,Albania,483.0,5.662090e+02,Albania,Albania
1,tt10223460,Albania,30528.0,3.228018e+04,Albania,Albania
2,tt1051906,Albania,3359.0,3.874881e+03,Albania,Albania
3,tt10665342,Albania,9906.0,1.047456e+04,Albania,Albania
4,tt10954984,Albania,4186.0,4.426259e+03,Albania,Albania
...,...,...,...,...,...,...
153294,tt9883996,World,6572790.0,7.582250e+06,World,World
153295,tt9896876,World,331328.0,3.713260e+05,World,World
153296,tt9907782,World,4588389.0,5.142300e+06,World,World
153297,tt9908390,World,3507860.0,4.046603e+06,World,World


In [15]:
bx_infl_new = bx_infl_new[["tconst", "long_name", "values", "inflated_values"]]

In [16]:
bx_infl_new

,tconst,long_name,values,inflated_values
0,tt7399138,Albania,483.0,5.662090e+02
1,tt10223460,Albania,30528.0,3.228018e+04
2,tt1051906,Albania,3359.0,3.874881e+03
3,tt10665342,Albania,9906.0,1.047456e+04
4,tt10954984,Albania,4186.0,4.426259e+03
...,...,...,...,...
153294,tt9883996,World,6572790.0,7.582250e+06
153295,tt9896876,World,331328.0,3.713260e+05
153296,tt9907782,World,4588389.0,5.142300e+06
153297,tt9908390,World,3507860.0,4.046603e+06


## Upload box_office_inflated

In [17]:
dataframes = {
    "box_office_data_inflated":bx_infl_new
}

In [18]:
# table_name = 'eu_merged'
engine = sqlf.get_engine()
engine
# Write records stored in a dataframe to SQL database
if engine != None:
    try:
        for table_name, df in dataframes.items():
            df.to_sql(table_name, # Name of SQL table
                            con=engine, # Engine or connection
                            if_exists='replace', # Drop the table before inserting new values 
                            schema=schema, # your class schema
                            index=False, # Write DataFrame index as a column
                            chunksize=5000, # Specify the number of rows in each batch to be written at a time
                            method='multi') # Pass multiple values in a single INSERT clause
            print(f"The {table_name} table was uploaded successfully.")
    # Error handling
    except (Exception, psycopg2.DatabaseError) as error:
        print(error)
        engine = None
else:
    print("nope") 

The box_office_data_inflated table was uploaded successfully.
